In [1]:
import pandas as pd
import numpy as np
import random
from citipy import citipy
import timeit
import requests
from config import api_key

In [2]:
lats=np.random.uniform(-90.000,90.000, size=1500)
lngs=np.random.uniform(-180.000,180.000, size=1500)
lat_lngs=zip(lats,lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)
len(coordinates)

1500

In [4]:
city="London"
url="http://api.openweathermap.org/data/2.5/weather?"
query_url= url + "appid=" + api_key + "&q=" + city
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=58bf6497f87428e2ca65f32a4e8f8b81&q=London'

In [5]:
resp= requests.get(query_url)
print(resp)
resp.json()

<Response [200]>


{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 290.97,
  'feels_like': 290.87,
  'temp_min': 287.99,
  'temp_max': 293.13,
  'pressure': 1027,
  'humidity': 79},
 'visibility': 10000,
 'wind': {'speed': 1.03, 'deg': 0},
 'clouds': {'all': 0},
 'dt': 1626565805,
 'sys': {'type': 2,
  'id': 2006068,
  'country': 'GB',
  'sunrise': 1626581075,
  'sunset': 1626638909},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [6]:
cities=[]
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    cities.append(city)
print(len(cities))

1500


In [20]:
# from datetime import datetime
# city_data = []
# selected_cities=cities[100:111]
# for city in selected_cities:
# #     print(city)
#     url="http://api.openweathermap.org/data/2.5/weather?"
#     query_url= url + "appid=" + api_key + "&q=" + city
#     resps=requests.get(query_url).json()
# #     print(resps)
#     try:
# #         print(resps["weather"][0]["main"])
#         country = resps["sys"]["country"]
#         lat = resps["coord"]["lat"]
#         lng = resps["coord"]["lon"]
#         max_temp = resps["main"]["temp_max"]
#         humidity = resps["main"]["humidity"]
#         clouds = resps["clouds"]["all"]
#         wind = resps["wind"]["speed"]
#         date = resps["dt"]
#         city_date=datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')
#         city_data.append({"City": city.title(),
#                           "Lat": lat,
#                           "Lng": lng,
#                           "Max Temp": max_temp,
#                           "Humidity": humidity,
#                           "Cloudiness": clouds,
#                           "Wind Speed": wind,
#                           "Country": country,
#                           "Date": city_date})       
#     except KeyError:
#         print(None)
        
# print(city_data)

[{'City': 'Waipawa', 'Lat': -41.4122, 'Lng': 175.5153, 'Max Temp': 286.5, 'Humidity': 75, 'Cloudiness': 100, 'Wind Speed': 6.5, 'Country': 'NZ', 'Date': '2021-07-18 00:42:06'}, {'City': 'Krutinka', 'Lat': 56.0055, 'Lng': 71.5113, 'Max Temp': 285.16, 'Humidity': 87, 'Cloudiness': 44, 'Wind Speed': 3.14, 'Country': 'RU', 'Date': '2021-07-18 00:42:07'}, {'City': 'Mataura', 'Lat': -46.1927, 'Lng': 168.8643, 'Max Temp': 280.64, 'Humidity': 93, 'Cloudiness': 100, 'Wind Speed': 0.56, 'Country': 'NZ', 'Date': '2021-07-18 00:42:07'}, {'City': 'East London', 'Lat': -33.0153, 'Lng': 27.9116, 'Max Temp': 286.64, 'Humidity': 55, 'Cloudiness': 0, 'Wind Speed': 2.88, 'Country': 'ZA', 'Date': '2021-07-18 00:42:15'}, {'City': 'Rikitea', 'Lat': -23.1203, 'Lng': -134.9692, 'Max Temp': 295.15, 'Humidity': 73, 'Cloudiness': 100, 'Wind Speed': 12.03, 'Country': 'PF', 'Date': '2021-07-18 00:42:15'}, {'City': 'Ferme-Neuve', 'Lat': 46.7001, 'Lng': -75.4493, 'Max Temp': 295.03, 'Humidity': 87, 'Cloudiness': 100

In [18]:
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
from datetime import datetime
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-07-18 00:05:05'

In [22]:
from datetime import datetime
city_data = []
for city in cities:
#     print(city)
    url="http://api.openweathermap.org/data/2.5/weather?"
    query_url= url + "appid=" + api_key + "&q=" + city
    resps=requests.get(query_url).json()
#     print(resps)
    try:
#         print(resps["weather"][0]["main"])
        country = resps["sys"]["country"]
        lat = resps["coord"]["lat"]
        lng = resps["coord"]["lon"]
        max_temp = resps["main"]["temp_max"]
        humidity = resps["main"]["humidity"]
        clouds = resps["clouds"]["all"]
        wind = resps["wind"]["speed"]
        date = resps["dt"]
        city_date=datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')
        city_data.append({"City": city.title(),
                          "Lat": lat,
                          "Lng": lng,
                          "Max Temp": max_temp,
                          "Humidity": humidity,
                          "Cloudiness": clouds,
                          "Wind Speed": wind,
                          "Country": country,
                          "Date": city_date})       
    except KeyError:
        pass 

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)       

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hermanus,-34.4187,19.2345,285.82,84,6,4.34,ZA,2021-07-18 00:42:09
1,Akyab,20.1500,92.9000,300.87,80,41,1.49,MM,2021-07-18 00:44:51
2,Ankazoabo,-22.2833,44.5167,289.85,64,6,2.29,MG,2021-07-18 00:44:51
3,Cam Ranh,11.9214,109.1591,301.13,74,20,0.51,VN,2021-07-18 00:44:51
4,Hermanus,-34.4187,19.2345,285.82,84,6,4.34,ZA,2021-07-18 00:42:09
5,Geraldton,-28.7667,114.6000,285.38,87,0,4.63,AU,2021-07-18 00:44:52
6,Bengkulu,-3.8004,102.2655,298.68,82,78,1.60,ID,2021-07-18 00:42:09
7,Puerto Ayora,-0.7393,-90.3518,295.12,93,94,2.68,EC,2021-07-18 00:44:52
8,Tasiilaq,65.6145,-37.6368,279.24,78,100,2.03,GL,2021-07-18 00:44:52
9,Bredasdorp,-34.5322,20.0403,285.53,87,7,0.00,ZA,2021-07-18 00:44:52


In [24]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377 entries, 0 to 1376
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        1377 non-null   object 
 1   Lat         1377 non-null   float64
 2   Lng         1377 non-null   float64
 3   Max Temp    1377 non-null   float64
 4   Humidity    1377 non-null   int64  
 5   Cloudiness  1377 non-null   int64  
 6   Wind Speed  1377 non-null   float64
 7   Country     1377 non-null   object 
 8   Date        1377 non-null   object 
dtypes: float64(4), int64(2), object(3)
memory usage: 96.9+ KB


In [26]:
new_columns_order=["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
city_data_df=city_data_df[new_columns_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Hermanus,ZA,2021-07-18 00:42:09,-34.4187,19.2345,285.82,84,6,4.34
1,Akyab,MM,2021-07-18 00:44:51,20.1500,92.9000,300.87,80,41,1.49
2,Ankazoabo,MG,2021-07-18 00:44:51,-22.2833,44.5167,289.85,64,6,2.29
3,Cam Ranh,VN,2021-07-18 00:44:51,11.9214,109.1591,301.13,74,20,0.51
4,Hermanus,ZA,2021-07-18 00:42:09,-34.4187,19.2345,285.82,84,6,4.34


In [29]:
output_data_file = "./cities.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")